# Multi-layer perceptron (Deep Learning) with Keras / Tensorflow

Please note - Tensorflow and PyTorch should not be imported together into your virtual environment, as they are known to be incompatible, due to using similar CUDA drivers. Always create a new virtual environment. 

In [ ]:
#!pip install tensorflow pandas scikit-learn modelviz

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from deep_learn.metrics import compute_metrics
import pandas as pd
from modelviz.confusion_matrix import plot_confusion_matrix

## Load in data

In [ ]:
mlp_data = pd.read_csv('data/synthetic_income_data.csv', 
                       index_col=None)

In [ ]:
X = mlp_data.drop(columns=['income'])
y = mlp_data['income']

## Prepare train and test set splits

In [ ]:
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
# Create train and test splits
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_SEED
)
# Create train and validation splits
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=1/9, random_state=RANDOM_SEED
)


## Scale the data

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

## Define Tensorflow Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import Input

In [ ]:
model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)  
])

## Compile the model

In [ ]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_squared_error'])

## Add Early stopping

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True)

## Add model checkpointing

In [ ]:
checkpoint_path = "models/keras_best_model.keras"
checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,  
    verbose=1)

## Fit the model

In [ ]:
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, checkpoint_cb],  
    verbose=1)


## Evaluation process

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model("models/keras_best_model.keras")

In [ ]:
# Predict on test data
y_pred = best_model.predict(X_test_scaled).flatten()

In [ ]:
compute_metrics(y_test, y_pred)

## Visualize evaluation metrics

In [ ]:
from modelviz.regression import regression_diagnostics_panel

regression_diagnostics_panel(
    y_test=y_test,
    y_pred=y_pred,
    qq_line_color='grey', qq_point_color='black'
)